### Postać rozwiązania

Nasze rozwiązanie będzie miało postać wektorów wierzchołków grafu ułożonych w kolejności odwiedzania przypisanych do poszczególnych dronów:<br>
<center>
$d_{1}=(1,3,5...)$<br>
$d_{2}=(2,7,9...)$<br>
&emsp;&emsp;.<br>
&emsp;&emsp;.<br>
&emsp;&emsp;.<br>
$d_{M}=(10,15,4...)$<br>
   </center>
M - ilość wymaganych dronów

### Funkcja celu

\begin{equation*}
F=min (\sum \limits _{v=1} ^{M} \sum \limits _{i=0} ^{N} \sum \limits _{j=0} ^{N} c_{ij} x_{vij} + \alpha (M-D))
\end{equation*}

gdzie:
<ul>
    <li>$M$ – liczba dronów
    <li>$N$ – liczba odbiorców
    <li>$c_{ij}$ – koszt(czas) przejazdu od odbiorcy $i$ do odbiorcy $j$
    <li>$x_vij$ – zmienna decyzyjna która określa czy dron $v$ wykonuje trase od $i$ do $j$
    <li>$x_{vij} = \left\{
            \begin{array}{ll}
                1 & \mbox{gdy realizowany jest kurs od i do j przez drona v}\\
                0 & \mbox{w przeciwnym przypadku}
            \end{array}
        \right. 
        $
    <li>$\alpha$ – Współczynnik kary
    <li>$D$ – Liczba dronów w posiadaniu 
</ul>                  

### Ograniczenia
Jeśli $ x_{vij} = 1 \Rightarrow  u_{i} + q_{vj} = u_{j}$</br>

<center>$ q_{vi} \leq u_{i} \leq Q \quad \forall_{i} \in_{1, 2,..., N} $</center>

<!-- \begin{equation*}
\sum \limits _{v=1} ^{M} \sum \limits _{i=0} ^{N} y_{iv} \leq Q
\end{equation*}
\begin{equation*}
\sum \limits _{i=0} ^{N} y_{iv} \leq Q \quad }
\end{equation*}
 -->
$ q_{vi} $ – wielkość zapotrzebowania dostarczana do $i$-tego klienta przez $v$-tego drona

$Q $ – ładowność drona


### Struktury danych potrzebnych do algorytmu

> Macierz dystansów od poszczególnych odbiorców

\begin{equation*}
M_{i,j} = 
\begin{pmatrix}
d_{1,1} & d_{1,2} & \cdots & d_{1,n} \\
d_{2,1} & d_{2,2} & \cdots & d_{2,n} \\
\vdots  & \vdots  & \ddots & \vdots  \\
d_{i,1} & d_{i,2} & \cdots & d_{i,j} 
\end{pmatrix}
\end{equation*}

gdzie:
<ul>
    <li>$i, j$ - identyfikatory poszczególnych odbiorców</li>
    <li>$d_{i,j}$ - dystans pomiędzy odbiorcami $i, j$</li>
    <li>dystans - odległość euklidesowa $d_{i,j} = \sqrt{(x_{i}-x_{j})^{2}+(y_{i}-y_{j})^{2}}$
</ul>

> Lista położenia poszczególnych odbiorców

<center>$ L_{i} = a_{1}, a_{2},... a_{i} $</center>

gdzie:
<ul>
    <li>$a_{i}$ - koordynaty $i - tego$ odbiorcy $(x_{i}, y_{i})$</li>
</ul>


> Ilość dronów - M

> Ładowność drona - Q

> Ilość odbiorców - N




In [399]:
import numpy as np
import random
from matplotlib import pyplot as plt
import copy

M = 3
N = 40
Q = 4

class Client:
    """
    Odbiorca
        id: Identyfikator odbiorcy
        x, y: Koordynaty odbiorcy
    """
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y
    
    def __repr__(self):
#         return 'Client {}:  position: ({}, {})'.format(self.id, self.x, self.y)
        return 'Client ({}, {})'.format(self.x, self.y)
    
    def get_distance_from(self, point):
        return np.sqrt((point.x-self.x)**2 + (point.y-self.y)**2)
        

class Drone:
    """
    Dron
        id: Identyfikator drona
        x, y: Koordynaty drona
        velocity:  Prędkość drona 60km/h = 1km/min
        num_of_packages: Ilość paczek - Q
    """
    
    def __init__(self, id, capicity):
        self.id = id
        self.velocity = 60
        self.num_of_packages = capicity
        self.battery = 100
        self.temp_client_id = None
        self.x = 0
        self.y = 0
        self.x_client = None
        self.y_client = None
        self.x_prev_client = None
        self.x_prev_client = None
    
    def __repr__(self):
#         return 'Drone {}:  battery: {}%  position: ({}, {})  packages: {}'.format(self.id, self.battery,
#                                                                                   self.x, self.y, self.num_of_packages)
        return 'Drone {}'.format(self.id)
    
    def change_position(self, x, y):
        """
        Zmiana pozycji drona
        """
        self.x = x
        self.y = y
    
    def discharge(self):
        self.battery -= 1
    
    def charge(self):
        self.battery += 1
    
    def get_distance_from_client(self):
        """
        Odległość euklidesowa
        """
        return np.sqrt((self.x_client-self.x)**2 + (self.y_client-self.y)**2)
    
    def deliver_package(self):
        """
        Dostarczanie paczki
        """
        self.num_of_packages -= 1
        self.x_prev_client = self.x_client
        self.y_prev_client = self.y_client
        if self.num_of_packages == 0:
            self.temp_client_id = None
            self.x_client = 0
            self.y_client = 0
        else:
            self.temp_client_id = None
            self.x_client = None
            self.y_client = None

    def load_packages(self):
        """ 
        Załadowanie drona
        """
        self.num_of_packages = Q
    
    def specify_client(self, client):
        """
        Przypisanie klienta do drona
        """
        if (self.temp_client_id == None):
            self.temp_client_id = client.id
            self.x_client = client.x
            self.y_client = client.y
    
    def travel(self):
        """
        Przemieszczanie się drona
        """
        distance = self.get_distance_from_client()
        if (distance <= 1):
            if self.x_client == 0 and self.y_client == 0:
                self.load_packages()
            self.x = self.x_client
            self.y = self.y_client
            self.deliver_package()
        else:
            w = abs(self.x - self.x_client)
            h = abs(self.y - self.y_client)
            sin_alpha = h/distance
            sin_beta = w/distance
            if (self.x >= self.x_client and self.y >= self.y_client):
                self.change_position(self.x - sin_beta, self.y - sin_alpha)
            if (self.x <= self.x_client and self.y <= self.y_client):
                self.change_position(self.x + sin_beta, self.y + sin_alpha)
            if (self.x >= self.x_client and self.y <= self.y_client):
                self.change_position(self.x - sin_beta, self.y + sin_alpha)
            if (self.x <= self.x_client and self.y >= self.y_client):
                self.change_position(self.x + sin_beta, self.y - sin_alpha)
        self.discharge()

class TabuSearch:
    """
    TabuSearch
        drones: Wszyskie dostępne drony. Ilość - M
        clients: Nieodwiedzeni odbiorcy
        x_visited, y_visited: Koordynaty odwiedzonych odbiorców
        total_time: Całkowity czas dostarczania paczek
    """
    def __init__(self, drones, clients, solution):
        self.drones = drones
        self.clients = clients
        self.total_time = 0
        
        # Potrzebne do wizualizacji
        self.solution = solution
        self.x_clients = []
        self.y_clients = []
        self.x_visited = []
        self.y_visited = []
        self.x_drones = []
        self.y_drones = []
    
    def initialize_client_positions(self):
        """
        Inicjalizacja pozycji odbiorców do dronów
        """
        for client in self.clients:
            self.x_clients.append(client.x)
            self.y_clients.append(client.y)
    
    def update_drone_positions(self):
        """
        Aktualizacja pozycji dronów
        """
        self.x_drones = []
        self.y_drones = []
        for drone in self.drones:
            self.x_drones.append(drone.x)
            self.y_drones.append(drone.y)
    
    def update_visited_clients(self, x, y):
        """
        Aktualizacja odwiedzonych odbiorców
        """
        self.x_visited.append(x)
        self.y_visited.append(y)
    
    def assign_clients(self):
        """
        Przipisanie odbiorców do wszystkich dronów
        """
        for drone in self.drones:
            if self.solution[drone]:
                drone.specify_client(self.solution[drone].pop(0))
    
    def assign_client(self, drone_id):
        """
        Przipisanie odbiorcy do jednego drona o przekazanym ID
        """
        for drone in self.drones:
            if drone.id == drone_id:
                if self.solution[drone]:
                    drone.specify_client(self.solution[drone].pop(0))

In [400]:
# Need for figure outside jupyter
%matplotlib qt

#### Tworzenie dronów

In [401]:
drones = []
for i in range(M):
    d = Drone(i+1, Q)
    drones.append(d)

#### Tworzenie odbiorców z losowymi koordynatami

In [402]:
# Creating clients with random (x, y) positions
clients = []
for i in range(N):
    x_pos = 0
    y_pos = 0
    while (y_pos == 0 and x_pos == 0):
        x_pos = random.randint(-25,25)
        y_pos = random.randint(-25,25)
    c = Client(i+1, x_pos, y_pos)
    clients.append(c)
clients_copy = copy.deepcopy(clients)

#### Tworzenie macierzy kosztów

In [403]:
cost_matrix = np.zeros((len(clients)+1, len(clients)+1))
base = Client(0,0,0)

for i in range(len(clients) + 1):
    for j in range(i, len(clients) + 1):
        if i != j:
            if i == 0:
                cost_matrix[i][j] = clients[j-1].get_distance_from(base)
                cost_matrix[j][i] = clients[j-1].get_distance_from(base)
            else:
                cost_matrix[i][j] = clients[i-1].get_distance_from(clients[j-1])
                cost_matrix[j][i] = clients[i-1].get_distance_from(clients[j-1])
                
print(cost_matrix)


[[ 0.         19.20937271 21.02379604 ... 23.76972865 10.77032961
  12.04159458]
 [19.20937271  0.         39.96248241 ... 14.2126704   9.43398113
  16.        ]
 [21.02379604 39.96248241  0.         ... 40.60788101 31.78049716
  27.65863337]
 ...
 [23.76972865 14.2126704  40.60788101 ...  0.         19.41648784
  13.03840481]
 [10.77032961  9.43398113 31.78049716 ... 19.41648784  0.
  13.60147051]
 [12.04159458 16.         27.65863337 ... 13.03840481 13.60147051
   0.        ]]


#### Generating first solution

In [404]:
dic = {}
for d in drones:
    dic[d] = [clients.pop(random.randint(0, len(clients)-1)) for _ in range(d.num_of_packages)]
    dic[d].insert(0, base)

def find_next_path(drones, paths):
    lowest = 100*(2+Q-1)
    idx = 0
    for i, d in enumerate(drones):
        s = sum(cost_matrix[paths[d][i].id][paths[d][i+1].id] for i in range(len(paths[d])-1))
        if s <= lowest:
            idx = i
            lowest = s
    return idx

while clients:
    drone_idx = find_next_path(drones, dic)
    if base in dic[drones[drone_idx]][-Q:]:
        dic[drones[drone_idx]].append(clients.pop(random.randint(0, len(clients)-1)))
    else:
        dic[drones[drone_idx]].append(base)


In [405]:
# Initialization and assignment of structures 
ts1 = TabuSearch(drones, clients_copy, dic)
ts1.initialize_client_positions()
ts1.update_drone_positions()
ts1.assign_clients()

In [406]:
fig = plt.figure(figsize=(12,12))

In [407]:
print(dic)

{Drone 1: [Client (15, -22), Client (18, 2), Client (9, 15), Client (-13, 8), Client (0, 0), Client (25, -25), Client (-22, -14), Client (-6, 11), Client (-24, -8), Client (0, 0), Client (25, 22), Client (15, -3), Client (-23, -6), Client (5, 9), Client (0, 0), Client (-8, 16)], Drone 2: [Client (9, 19), Client (0, 12), Client (-22, -6), Client (-16, -9), Client (0, 0), Client (19, -13), Client (0, -13), Client (-18, 3), Client (25, -20), Client (0, 0), Client (-8, 17), Client (8, 22), Client (-12, -15), Client (20, -7), Client (0, 0), Client (-2, -15), Client (-17, 0), Client (-12, -9)], Drone 3: [Client (-4, -10), Client (-25, -7), Client (8, 24), Client (-12, 1), Client (0, 0), Client (24, 11), Client (3, 17), Client (13, 16), Client (23, -4), Client (0, 0), Client (-18, -3), Client (23, -22), Client (-23, -12), Client (23, -24), Client (0, 0)]}


#### Wizualizacja dostarczania paczek poprzez losowe przypisywanie odbiorców do dronów

In [408]:
ts_copy = copy.deepcopy(ts1)
k = 1
for i in range(1000):
    if k == 4:
        k = 1
    else:
        k = k+1
    for drone in ts_copy.drones:
        if drone.temp_client_id == None:
            ts_copy.assign_client(drone.id)
            ts_copy.update_visited_clients(drone.x_prev_client, drone.y_prev_client)
#         if drone.temp_client_id:
        drone.travel()
    ts_copy.update_drone_positions()
    plt.plot(ts_copy.x_clients, ts_copy.y_clients, 'go', markersize=12, label="Odbiorca")
    plt.plot(ts_copy.x_drones, ts_copy.y_drones, 'm{}'.format(k), markersize=20, label="Dron")
    plt.plot(ts_copy.x_visited, ts_copy.y_visited, 'ro', markersize=12, linewidth=4, label="Dostarczona paczka")
    plt.plot(ts_copy.x_visited, ts_copy.y_visited, 'wx', markersize=12)
    plt.plot(0, 0, 'bo-', markersize=14)
    plt.grid()
    plt.ylim(-30, 30)
    plt.xlim(-30, 30)
    plt.title(f'Akutalny czas dostarczania paczek w minutach: {i+1}')
    plt.legend()
    plt.draw()
    plt.pause(0.002)
    plt.cla()
plt.show()

KeyboardInterrupt: 

KeyboardInterrupt: 